#This note book traces the path that the data would take in order to pass it into the neural network evaluator

In [1]:
#These are all your imports

import pandas as pd
import numpy as np
import csv

In [ ]:
#The first step in making the neural network is converting the training files into a format that can be read by the neural network FOR TRAINING.
#this code block takes a csv training catalog and loads all the files into one NN readable file.


#PLEASE NOTE - we recomend running the mSEED file converter not this one. Both work but this one is roughly half as fast

#this is where you specify your catalog URL
cat_file_url = './space_apps_2024_seismic_detection/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv'
#this is the folder that holds the csv files mentioned in your catalog
training_data_directory = './space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/'



#this code segment is for converting csv files to a type that can be read by the neural network

training_Catalog = pd.read_csv(cat_file_url)
csvfile = open('lunarOutputTest.csv', 'w', newline='')
outputWriter = csv.writer(csvfile)

correct_Times = np.array(training_Catalog['time_rel(sec)'].tolist())
data_Filenames = np.array(training_Catalog['filename'].tolist())
firstVelocityOfPreviousLoop = 1000
for i in range(0,len(data_Filenames)):
    local_Data = [
        correct_Times[i]
    ]
    print(i)
    local_Data_File = pd.read_csv(training_data_directory + data_Filenames[i] + '.csv')
    velocities = np.array(local_Data_File['velocity(m/s)'])


    #some files have multiple start points that mess up the training data this is used to keep track of the 
    #first index of each loop and compared to the next one to see if they are the same.
    #if they are the training data compiler will ignore the second file.
    if(len(velocities) < 572262 or firstVelocityOfPreviousLoop == velocities[0]):
        continue
    firstVelocityOfPreviousLoop = velocities[0]

    for j in range(0,572262):
        local_Data.append(velocities[j])
        if(j == len(velocities)-1):
            print(j)
    outputWriter.writerow(local_Data)
csvfile.close()


#The lunarOutputTest.csv file now can be inputed into the NN

In [ ]:
#this code segment is for converting mseed files to a type that can be read by the neural network
#it does the same thing as the code block above, so only one needs to be ran.
#once again this is for TRAINING DATA not test data

import time
from obspy import read
#sets the frequency bands for the filter
minfreq = .5
maxfreq = 1.0

#loading the csv files
training_Catalog = pd.read_csv(cat_file_url)
csvfile = open('lunarOutputTest.csv', 'w', newline='')
outputWriter = csv.writer(csvfile)

#pulls the times and filenames from the catalog to be used in the for loop
correct_Times = np.array(training_Catalog['time_rel(sec)'].tolist())
data_Filenames = np.array(training_Catalog['filename'].tolist())

#some files have multiple start points that mess up the training data this is used to keep track of the 
#first index of each loop and compared to the next one to see if they are the same.
#if they are the training data compiler will ignore the second file.

firstVelocityOfPreviousLoop = 1000
for i in range(0,len(data_Filenames)):
    local_Data = [
        correct_Times[i]
    ]
    print(i)

    mseed_file = training_data_directory + data_Filenames[i] + '.mseed'
    local_Data_File = None
    local_Data_File = read(mseed_file)
    #                takes file and applies bandpass filter from .5 to 1 hertz        pulls the velocities into an array
    velocities = local_Data_File.filter('bandpass', freqmin=minfreq, freqmax=maxfreq).traces[0].data

    #some files have multiple start points that mess up the training data this is used to keep track of the 
    #first index of each loop and compared to the next one to see if they are the same.
    #if they are the training data compiler will ignore the second file.

    
    if(len(velocities) < 572262 or velocities[0] == firstVelocityOfPreviousLoop):
        continue

    firstVelocityOfPreviousLoop = velocities[0]

    #
    for j in range(0,572263):
        local_Data.append(velocities[j])
    outputWriter.writerow(local_Data)


csvfile.close()


#The lunarOutputTest.csv file now can be inputed into the NN

#you can open the csv created in the previous step if you want to just be warned that it is very large.
the format is as follows

line 1: where the seismic activity starts, velocity1, velocity2, velocity3, ...
line 2: where the seismic activity starts, velocity1, velocity2, velocity3, ...
...

where each line represents one csv or mseed file

In [ ]:
#this code segment is for converting mseed files, specifically the test files (not the training files), to a type that can be read by the neural network
#this code outputs the exact same format as the last two the only difference is that we dont know when the seismic activity starts so we leave that zero.


import os
from obspy import read


#sets the frequency bands for the filter
minfreq = .5
maxfreq = 1.0

csvfile = open('lunarOutputTestApollo2_B.csv', 'w', newline='')
outputWriter = csv.writer(csvfile)


#find all the mseed files in the directory
folderUrl = './space_apps_2024_seismic_detection/data/lunar/test/data/S12_GradeB/'
data_Filenames = os.listdir(folderUrl)
for file in data_Filenames:
    if not (file.endswith('d')):
        data_Filenames.remove(file)



for i in range(0, len(data_Filenames)):
    local_Data = [
        0
    ]
    print(i)
    mseed_file = test_data_directory + data_Filenames[i]
    local_Data_File = read(mseed_file)
    #                takes file and applies bandpass filter from .5 to 1 hertz        pulls the velocities into an array
    velocities = local_Data_File.filter('bandpass', freqmin=minfreq, freqmax=maxfreq).traces[0].data

    #handles the corrupted small files that our ai cant read.
    if(len(velocities) < 572262):
        continue


    #
    for j in range(0,572263):
        if(j == 572262-1):
            print(j)
        local_Data.append(velocities[j])
    outputWriter.writerow(local_Data)

csvfile.close()

#you can open the csv created in the previous step if you want to just be warned that it is very large.
the format is as follows

line 1: 0, velocity1, velocity2, velocity3, ...
line 2: 0, velocity1, velocity2, velocity3, ...
...

where each line represents one csv or mseed file

#All this notebook does is convert files to a format usable by the NN the actual NN code is in the SpaceAppAI naotebook.